In [67]:
import pandas as pd
import numpy as np
import time
from time import mktime
from datetime import datetime, timedelta
import geopy.distance
import gc
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import data_prep_functions
import request_api_nba_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
base_ini = pd.read_csv("../bases_nba_stats/aux_files/base_nba_per_game_16_17.csv")

In [3]:
all_games_16_17 = data_prep_functions.prepara_base(base_ini)

In [4]:
all_games_16_17 = data_prep_functions.cria_features(all_games_16_17)

In [5]:
geral_16_17 = data_prep_functions.gera_last_N_games(all_games_16_17)

In [6]:
data_prep_functions.get_last_games(all_games_16_17, 
                                   geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1).DATE.iloc[0]
                                   , "IND")[["team_home", "team_away", "PTS_home", "PTS_away", 
                                             "FGA_home", "FGA_away", "fl_home_win"]]

,team_home,team_away,PTS_home,PTS_away,FGA_home,FGA_away,fl_home_win
GAME,,,,,,,
IND @ CLE 2017-04-02,CLE,IND,135,130,95,105,1
TOR @ IND 2017-04-04,IND,TOR,108,90,89,76,1
MIL @ IND 2017-04-06,IND,MIL,104,89,80,71,1
IND @ ORL 2017-04-08,ORL,IND,112,127,84,86,0
IND @ PHI 2017-04-10,PHI,IND,111,120,75,85,0


In [7]:
geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1)["FGA_home_L5"]

ATL @ IND 2017-04-12    89.0
Name: FGA_home_L5, dtype: float64

In [8]:
cross_16_17 = data_prep_functions.variaveis_delta(geral_16_17)

In [9]:
cross_16_17.to_csv("../bases_nba_stats/delta_cross_L5_nba_16_17.csv")

In [11]:
cross_16_17.fl_home_win.mean()

0.583739837398374

In [115]:
datetime.strptime(str(get_date_place("0021800270", year = "2018")[0]), '%Y-%m-%d')

datetime.datetime(2018, 11, 23, 0, 0)

In [9]:
games_ids = request_api_nba_stats.get_list_gameids(datetime.today()) 

In [14]:
df_full, df_full_jogo, erros = request_api_nba_stats.get_nba_stats_data(games_ids)

Length mismatch: Expected axis has 0 elements, new values have 28 elements
Length mismatch: Expected axis has 0 elements, new values have 28 elements


In [16]:
base_nba_2018 = pd.concat(df_full_jogo).reset_index()

In [17]:
base_nba_2018.to_csv("base_nba_2018_raw.csv")

In [22]:
all_games = data_prep_functions.prepara_base(base_nba_2018)

In [23]:
all_games = data_prep_functions.cria_features(all_games)

In [24]:
resp = data_prep_functions.gera_last_N_games(all_games, N=[5])

In [26]:
delta_L5_2018 = data_prep_functions.variaveis_delta(resp)

In [28]:
delta_L5_2018.to_csv("delta_cross_L5_nba_18_19.csv")

In [29]:
all_games.to_csv("all_games_nba_18_19.csv")

### Fluxo Predição no Dia

In [60]:
new_games_ids = request_api_nba_stats.get_list_gameids(max_date = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
                                                   , year = '2018', start_at=len(all_games))

In [73]:
new_games.empty

False

In [83]:
new_games = request_api_nba_stats.get_game_infos_dates(new_games_ids)

In [75]:
df_full, df_full_jogo, erros = request_api_nba_stats.get_nba_stats_data(new_games_ids)

In [76]:
new_games = pd.concat(df_full_jogo).reset_index()

In [77]:
new_games = data_prep_functions.prepara_base(new_games).reset_index()

In [91]:
new_games = data_prep_functions.cria_features(new_games, all_games)

In [93]:
to_predict = data_prep_functions.gera_last_N_games(new_games, all_games)

In [95]:
to_predict

,index,DATE,PLACE,team_home,team_away,DISTANCE_KM_home,DISTANCE_KM_away,DAYS_FROM_LAST_GAME_home,DAYS_FROM_LAST_GAME_away,FGM_home_L5,...,N_GAMES_AWAY_L4_days_away_L5_AWAY,N_GAMES_AWAY_L6_days_away_L5_AWAY,N_GAMES_AWAY_L8_days_away_L5_AWAY,N_GAMES_AWAY_L10_days_away_L5_AWAY,SUM_DIST_KM_away_L5_AWAY,AVG_DIST_KM_away_L5_AWAY,BACK_TO_BACK_away_L5_AWAY,AVG_DAYS_FROM_LG_away_L5_AWAY,STD_DAYS_FROM_LG_away_L5_AWAY,MIN_DAYS_FROM_LG_away_L5_AWAY
ORL @ SAS 2018-11-04,0,2018-11-04 00:00:00,AT&T Center,SAS,ORL,0,1663.41,1,2,42.4,...,0.0,0.0,0.0,0.0,4890.564204,1630.188068,0.0,1.666667,0.471405,1.0
MIN @ POR 2018-11-04,1,2018-11-04 00:00:00,Moda Center,POR,MIN,0,2294.94,1,2,40.2,...,0.0,0.0,0.0,0.0,6823.280027,1705.820007,0.0,0.000000,0.000000,0.0
TOR @ LAL 2018-11-04,2,2018-11-04 00:00:00,Staples Center,LAL,TOR,1329.52,3504.19,1,2,44.8,...,0.0,0.0,0.0,0.0,4300.093938,1433.364646,0.0,2.333333,0.942809,1.0
MIA @ DET 2018-11-05,3,2018-11-05 00:00:00,Little Caesars Arena,DET,MIA,743.675,1897.09,2,2,36.6,...,0.0,0.0,0.0,0.0,3834.028333,958.507083,0.0,0.000000,0.000000,0.0
HOU @ IND 2018-11-05,4,2018-11-05 00:00:00,Bankers Life Fieldhouse,IND,HOU,0,1392.67,2,2,41.2,...,0.0,0.0,0.0,0.0,8224.335454,2056.083863,0.0,2.000000,1.000000,1.0
CLE @ ORL 2018-11-05,5,2018-11-05 00:00:00,Amway Center,ORL,CLE,1663.41,1437.8,1,2,30.4,...,0.0,0.0,0.0,0.0,2206.627104,551.656776,0.0,0.000000,0.000000,0.0
CHI @ NYK 2018-11-05,6,2018-11-05 00:00:00,Madison Square Garden,NYK,CHI,330.844,1151.12,1,2,32.8,...,0.0,0.0,0.0,0.0,4262.042310,1065.510578,0.0,0.000000,0.000000,0.0
NOP @ OKC 2018-11-05,7,2018-11-05 00:00:00,Chesapeake Energy Arena,OKC,NOP,1855.15,926.626,3,2,43.6,...,0.0,0.0,0.0,0.0,9459.565630,1891.913126,0.0,0.000000,0.000000,0.0
BOS @ DEN 2018-11-05,8,2018-11-05 00:00:00,Pepsi Center,DEN,BOS,0,2849.26,2,2,43.0,...,0.0,0.0,0.0,0.0,5702.959074,1140.591815,0.0,2.200000,0.748331,1.0
TOR @ UTA 2018-11-05,9,2018-11-05 00:00:00,Vivint Smart Home Arena,UTA,TOR,597.057,2687.29,2,3,40.8,...,0.0,0.0,0.0,0.0,4300.093938,1433.364646,0.0,2.333333,0.942809,1.0


In [96]:
to_predict = data_prep_functions.variaveis_delta(to_predict)

In [97]:
to_predict[['D2_PLUS_MINUS_L5', 'D1_WIN_PCT_L5']]

,D2_PLUS_MINUS_L5,D1_WIN_PCT_L5
ORL @ SAS 2018-11-04,5.4,0.6
MIN @ POR 2018-11-04,12.4,0.2
TOR @ LAL 2018-11-04,0.0,-0.2
MIA @ DET 2018-11-05,2.0,-0.2
HOU @ IND 2018-11-05,3.4,0.4
CLE @ ORL 2018-11-05,-3.4,0.0
CHI @ NYK 2018-11-05,8.8,0.4
NOP @ OKC 2018-11-05,6.2,0.8
BOS @ DEN 2018-11-05,9.8,0.0
TOR @ UTA 2018-11-05,-3.4,-0.4
